In [89]:
import pandas as pd
import re
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
# word2vec packages
import gensim.downloader as api
from nltk import sent_tokenize
from gensim.utils import simple_preprocess
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm
import gensim

In [90]:
texts = pd.read_csv("SMSSpamCollection.txt", sep="\t", names=["label","message"])

In [91]:
texts["message"].loc[23]

'Aft i finish my lunch then i go str down lor. Ard 3 smth lor. U finish ur lunch already?'

In [92]:
texts.shape

(5572, 2)

In [25]:
### Download stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/cgiuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [26]:
## Stemer
ps = PorterStemmer()

### Clean the corpus: Text preprocessing

In [27]:
corpus= []

for i in range(0,len(texts)):
    review = re.sub('[^a-zA-Z]', ' ', texts["message"][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)
    
    

In [28]:
corpus[:20]

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
 'ok lar joke wif u oni',
 'free entri wkli comp win fa cup final tkt st may text fa receiv entri question std txt rate c appli',
 'u dun say earli hor u c alreadi say',
 'nah think goe usf live around though',
 'freemsg hey darl week word back like fun still tb ok xxx std chg send rcv',
 'even brother like speak treat like aid patent',
 'per request mell mell oru minnaminungint nurungu vettam set callertun caller press copi friend callertun',
 'winner valu network custom select receivea prize reward claim call claim code kl valid hour',
 'mobil month u r entitl updat latest colour mobil camera free call mobil updat co free',
 'gonna home soon want talk stuff anymor tonight k cri enough today',
 'six chanc win cash pound txt csh send cost p day day tsandc appli repli hl info',
 'urgent week free membership prize jackpot txt word claim c www dbuk net lccltd pobox ldnw rw',
 'search right word thank breather

## Bag of Words Model

In [29]:
cv = CountVectorizer(max_features=2500)
X = cv.fit_transform(corpus).toarray()

In [30]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [34]:
## One hot encode dependent variable
y=pd.get_dummies(texts['label'])
y=y.iloc[:,1].values

In [32]:
y

array([0, 0, 1, ..., 0, 0, 0], dtype=uint8)

Train test split

In [37]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 57)


In [39]:
spam_detect_model = MultinomialNB().fit(X_train, y_train)

In [40]:
spam_detect_model

MultinomialNB()

In [41]:
#prediction
y_pred=spam_detect_model.predict(X_test)

In [44]:
score=accuracy_score(y_test,y_pred)
print(score)

0.97847533632287


In [45]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99       960
           1       0.94      0.90      0.92       155

    accuracy                           0.98      1115
   macro avg       0.96      0.95      0.95      1115
weighted avg       0.98      0.98      0.98      1115



## Word2Vec Model

In [50]:
#!pip install gensim

In [48]:
# load googles word2vec model
wv = api.load('word2vec-google-news-300')

In [93]:
lemmatizer=WordNetLemmatizer()

In [94]:
corpus = []
for i in range(0, len(texts)):
    review = re.sub('[^a-zA-Z]', ' ', texts['message'][i])
    review = review.lower()
    review = review.split()
    
    review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [95]:
corpus[0]

'go jurong point crazy available bugis n great world la e buffet cine got amore wat'

In [96]:
words=[]
for sent in corpus:
    sent_token=sent_tokenize(sent)
    for sent in sent_token:
        words.append(simple_preprocess(sent))

In [97]:
words[0]

['go',
 'jurong',
 'point',
 'crazy',
 'available',
 'bugis',
 'great',
 'world',
 'la',
 'buffet',
 'cine',
 'got',
 'amore',
 'wat']

In [98]:
len(words)

5564

In [99]:
model = gensim.models.Word2Vec(words,window=5,min_count=2)

In [100]:
model.corpus_count

5564

In [101]:
model.epochs

5

In [102]:
model.wv.similar_by_word('meeting')

[('work', 0.9984989762306213),
 ('even', 0.9984346628189087),
 ('wat', 0.9983864426612854),
 ('im', 0.9983823299407959),
 ('yet', 0.9983537197113037),
 ('buy', 0.9983491897583008),
 ('money', 0.9983407258987427),
 ('right', 0.9983381032943726),
 ('back', 0.9983358383178711),
 ('sorry', 0.9983318448066711)]

In [103]:
model.wv.similar_by_word('work')

[('like', 0.9997450709342957),
 ('know', 0.9997392296791077),
 ('back', 0.99972003698349),
 ('come', 0.9997169971466064),
 ('time', 0.9997125267982483),
 ('said', 0.999695360660553),
 ('need', 0.9996944069862366),
 ('still', 0.999691367149353),
 ('going', 0.9996857047080994),
 ('go', 0.9996834993362427)]

In [104]:
model.wv['kid'].shape

(100,)

In [105]:
def avg_word2vec(doc):
    # remove out-of-vocabulary words
    #sent = [word for word in doc if word in model.wv.index_to_key]
    #print(sent)
    
    return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key],axis=0)
                #or [np.zeros(len(model.wv.index_to_key))], axis=0)

In [106]:
words[73]

['performed']

In [107]:
words[30]

['wait', 'still', 'clear', 'sure', 'sarcastic', 'want', 'live']

In [108]:
words[90]

['yeah', 'stand', 'close', 'tho', 'catch', 'something']

In [109]:
type(model.wv.index_to_key)

list

In [110]:
#apply for the entire sentences
X=[]
for i in tqdm(range(len(words))):
#     print("Hi there",i)
    X.append(avg_word2vec(words[i]))


100%|██████████| 5564/5564 [00:01<00:00, 5493.62it/s]


In [111]:
X_new=np.array(X)

/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [112]:
X_new[3]

array([-1.41740933e-01,  3.28980118e-01,  1.66340351e-01,  4.61039096e-02,
        1.05386376e-01, -3.90552759e-01,  8.74172226e-02,  6.33390069e-01,
       -2.15241134e-01, -1.19476676e-01, -1.88838065e-01, -4.60348636e-01,
       -3.54975499e-02,  1.87713683e-01,  1.43454269e-01, -2.98695683e-01,
        2.64999215e-02, -4.15612251e-01,  4.15876396e-02, -5.39947152e-01,
        1.52854577e-01,  1.77938268e-01,  1.15594469e-01, -1.60323516e-01,
       -1.25253186e-01,  2.37426162e-02, -2.14437366e-01, -2.00660810e-01,
       -3.15333277e-01,  5.50206900e-02,  4.09245580e-01,  6.99922740e-02,
        1.72828794e-01, -2.88164854e-01, -1.85448006e-01,  3.51725370e-01,
       -9.55177378e-03, -2.65880316e-01, -1.95817396e-01, -5.15341520e-01,
       -1.57512914e-04, -2.60490060e-01, -8.29297975e-02,  7.07969442e-02,
        3.07807505e-01, -1.35636434e-01, -2.57891685e-01,  9.01011750e-03,
        1.50920808e-01,  2.58330911e-01,  1.96551666e-01, -3.37062269e-01,
       -7.17410967e-02, -